In [1]:
# defining functions
import json 
import math

def getmeasurementTimestamp(item):
    return int(item['measurementTimestamp'])

def getProcessingTimestamp(item):
    return int(item['processingTimestamp'])

def get_x_error(item): #the error in the data is the stdev of the sample, we need the error of the estimation (the sample mean)
    return item['value']['averagecoordinate']['error']['coordinates'][0]/math.sqrt(item['value']['trackeeHistory']['nMeasurements'])

def get_y_error(item):
    return item['value']['averagecoordinate']['error']['coordinates'][1]/math.sqrt(item['value']['trackeeHistory']['nMeasurements'])

def get_fitted(item):
     return item['value']['trackeeHistory']['fitStatus']
    

In [2]:
#reading data
for i in [0]:
    data = []
    with open("arena/2015-07-05.json") as f:
        data = f.readlines()

    json_lines = []
    mac_adresses = []
    
    for line in data:
        jsline = json.loads(line)
        json_lines.append(jsline)#now json_lines contains all lines of data
        mac_adresses.append(jsline["value"]["sourceMac"]) # mac_addresses is a list of address per line
        


In [3]:


json_lines.sort(key = getmeasurementTimestamp) # now json_lines is sorted by time
minTime = getmeasurementTimestamp(json_lines[0])
maxTime = getmeasurementTimestamp(json_lines[len(json_lines) - 1])
print("minTime="+ str(minTime))
print("maxTime="+ str(maxTime))

timeMinutes = (maxTime - minTime)/1000/60

for i in [0]:
    average_x_error = 0
    average_y_error = 0
    count = 0
    good_lines = []
    number_of_people =  len(set(mac_adresses)) #number of mac adresses detected. Not people, because of MAC randomization
    mc_good = []
    delay = 0
    number_of_fitted = 0
    for jsline in json_lines:
        average_x_error += get_x_error(jsline)
        average_y_error += get_y_error(jsline)
        thisdelay = getProcessingTimestamp(jsline) - getmeasurementTimestamp(jsline)
        delay += thisdelay
        if (get_fitted(jsline) == 'FITTED'):
            number_of_fitted +=1
        if get_x_error(jsline) <= 2 and get_y_error(jsline) <= 2:             
            good_lines.append(jsline)
            mc_good.append(jsline["value"]["sourceMac"])

    number_of_lines = len(json_lines)
    number_of_people_good = len(set(mc_good)) # number of addresses where error < 2m 
    average_x_error /= number_of_lines
    average_y_error /= number_of_lines
    delay /= number_of_lines
    
    print ('time in minutes:' + str(timeMinutes)) 
    print ('number of lines: ' + str(number_of_lines))
    print ('average x error: ' + str(average_x_error))
    print ('average y error: ' + str(average_y_error))    
    print('number of people the whole day:' + str(number_of_people))     
    print('number of people with sdev < 2m:' + str(number_of_people_good))     
    print('average delay in ms:' + str(delay))
    print('number of fitted:' + str(number_of_fitted)) 

minTime=1436047367297
maxTime=1436133804311
time in minutes:1440.6169
number of lines: 772867
average x error: 3342.886813949892
average y error: 3697.2146526257307
number of people the whole day:30344
number of people with sdev < 2m:17736
average delay in ms:10034.32547644032
number of fitted:772867


In [4]:
concertFinishedTimestamp=minTime+1000*60*60*6#timestamp until 6 am

In [28]:
16170+5777

21947

In [5]:
FirstTimeSeen=dict()
LastTimeSeen=dict()
AddressesInSec = []
for jsline in json_lines:
    address = jsline["value"]["sourceMac"]
    time =  getmeasurementTimestamp(jsline)
    if address in FirstTimeSeen:        
        if time < FirstTimeSeen[address]:
            FirstTimeSeen[address] = time
    else:
        FirstTimeSeen[address] = time
        
    if address in LastTimeSeen:         
        if time > LastTimeSeen[address]:
            LastTimeSeen[address] = time
    else:
        LastTimeSeen[address] = time

In [9]:
DwellTime = dict()
numberOfPeopleAtConcert=0
for address in LastTimeSeen.keys():    
    DwellTime[address] = int((LastTimeSeen[address] - FirstTimeSeen[address]) /1000/60/10) # in 10 minutes interval, i.e. minutes/10
    if LastTimeSeen[address] < concertFinishedTimestamp:
        numberOfPeopleAtConcert += 1
print('number of addresses detected during concert hours:')
print(numberOfPeopleAtConcert)
longTermAddresses=[]
numberOfAddresses = []
for jsline in json_lines:
    sec = int(math.floor((getmeasurementTimestamp(jsline)- minTime)/1000))
    #print(str(sec))
    address = jsline["value"]["sourceMac"]
    if DwellTime[address] > 1:#  i.e. >30 minutes 
        longTermAddresses.append(address)
        if len(AddressesInSec) <= sec:
            while len(AddressesInSec) <= sec:
                AddressesInSec.append([])
                #numberOfAddresses.append(0)
            AddressesInSec[sec].append(address)
            #numberOfAddresses[sec] +=1
            
        else:
            if address not in AddressesInSec[sec]:                                
                AddressesInSec[sec].append(address)
                #numberOfAddresses[sec] +=1
for setje in AddressesInSec:
    numberOfAddresses.append(len(setje)) 
longTermCount = len(set(longTermAddresses))                
print("Long term persistent addresses (>10 min)= " + str(longTermCount))
average = 0            
maxN = 0
for addresses in AddressesInSec:            
    average += len(addresses)
    maxN = max(len(addresses), maxN)
average /= len(AddressesInSec)   
print("averageNumberOfAddressesPerSecondAtConcert " + str(average))
print(maxN)
print(len(numberOfAddresses))

number of addresses detected during concert hours:
25687
Long term persistent addresses (>10 min)= 9884
averageNumberOfAddressesPerSecondAtConcert 4.554085008908119
35
86438


In [ ]:
import matplotlib.pyplot as plt
plt.plot(numberOfAddresses)
plt.ylabel('addresses present')
plt.xlabel('sec')
axes = plt.gca()
axes.set_xlim([0,25000])
axes.set_ylim([0,max(numberOfAddresses)])
plt.show()

In [36]:
n_people_staying_this_much = []
maxDwellTime = max(DwellTime.values())
print("maxDwellTime=" + str(maxDwellTime))
for i in range(maxDwellTime + 1):   
    n_people_staying_this_much.append(0)
    
for address in DwellTime.keys():  
    if LastTimeSeen[address] < concertFinishedTimestamp:
        n_people_staying_this_much[DwellTime[address]] +=1
    
n_people_staying_less_than_half_hour =  n_people_staying_this_much[0] +   n_people_staying_this_much[1] + n_people_staying_this_much[2]
n_people_staying_less_than_10min =  n_people_staying_this_much[0]

n_people_staying_more_than_half_hour = numberOfPeopleAtConcert - n_people_staying_less_than_half_hour

print("number of people staying less than 30 min during concert hours  = " + str(n_people_staying_less_than_half_hour))
print("number of people staying less than 10 min during concert hours  = " + str(n_people_staying_less_than_10min))
print("number of people staying more than 30 min during concert hours = " + str(n_people_staying_more_than_half_hour))
percentage_people_staying_more_than_30_minutes = n_people_staying_more_than_half_hour/37000 * 100
print('percentage_people_staying_more_than_30_minutes = ' +  str(percentage_people_staying_more_than_30_minutes))

maxDwellTime=141
number of people staying less than 30 min during concert hours  = 18419
number of people staying less than 10 min during concert hours  = 17343
number of people staying more than 30 min during concert hours = 7268
percentage_people_staying_more_than_30_minutes = 19.643243243243244


In [20]:
import matplotlib.pyplot as plt
plt.plot(n_people_staying_this_much)
plt.ylabel('number of people 05 july')
plt.xlabel('dwell time in minutes/10')

plt.show()

NameError: name 'n_people_staying_this_much' is not defined

In [18]:
106/8

13.25